In [1]:
import copy, numpy as np

In [2]:
def sigmoid(x):
    # return 1.0 / (1.0 + np.exp(-x)) if x >= 0 else np.exp(x) / (1.0 + np.exp(x))
    return np.where(x >= 0, 1.0 / (1.0 + np.exp(-x)), np.exp(x) / (1.0 + np.exp(x)))


def sigmoid_derivative(d_output):
    return d_output * (1 - d_output)

In [3]:
binary_dim = 8

largest_number = pow(2, binary_dim)
int2binary = np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T, axis=1)

In [18]:
input_dim = 2
hidden_dim = 16
output_dim = 1
learning_rate = 0.1

synapse_ih = 2 * np.random.random((input_dim, hidden_dim)) - 1  # input_hidden
synapse_ho = 2 * np.random.random((hidden_dim, output_dim)) - 1  # hidden_output
synapse_hh = 2 * np.random.random((hidden_dim, hidden_dim)) - 1  # hidden_next_hidden

synapse_ih_update = np.zeros_like(synapse_ih)
synapse_ho_update = np.zeros_like(synapse_ho)
synapse_hh_update = np.zeros_like(synapse_hh)

error_count = 0
for epoch in range(10000):
    a_int = np.random.randint(largest_number / 2)
    a = int2binary[a_int]
    b_int = np.random.randint(largest_number / 2)
    b = int2binary[b_int]
    c_int = a_int + b_int
    c = int2binary[c_int]

    d = np.zeros_like(c)
    overall_error = 0


    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))

    for position in range(binary_dim):
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        layer_1 = sigmoid(np.dot(X, synapse_ih) + np.dot(layer_1_values[-1], synapse_hh))
        layer_2 = sigmoid(np.dot(layer_1, synapse_ho))

        layer_2_error = y - layer_2
        layer_2_deltas.append(layer_2_error * sigmoid_derivative(layer_2))
        overall_error += np.abs(layer_2_error[0])
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        layer_1_values.append(copy.deepcopy(layer_1))

    future_layer_1_delta = np.zeros(hidden_dim)
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position - 1]
        prev_layer_1 = layer_1_values[-position - 2]

        layer_2_delta = layer_2_deltas[-position - 1]
        layer_1_delta = (future_layer_1_delta.dot(synapse_hh.T) + layer_2_delta.dot(synapse_ho.T)) * sigmoid_derivative(
            layer_1)

        synapse_ho_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_hh_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_ih_update += X.T.dot(layer_1_delta)

        future_layer_1_delta = layer_1_delta

    synapse_ih += synapse_ih_update * learning_rate
    synapse_ho += synapse_ho_update * learning_rate
    synapse_hh += synapse_hh_update * learning_rate

    synapse_ih_update *= 0
    synapse_ho_update *= 0
    synapse_hh_update *= 0

    out = 0
    for index, x in enumerate(reversed(d)):
        out += x * pow(2, index)
    if out != c_int:
        error_count += 1

    # print out progress
    if epoch % 1000 == 0:
        print("Error:" + str(overall_error))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))

        print("in these 1000 epochs, there's %4d error " % error_count)
        error_count = 0
        print("------------")


Error:[3.86409399]
Pred:[0 0 0 0 0 0 0 1]
True:[1 1 0 0 0 1 0 1]
111 + 86 = 1
in these 1000 epochs, there's    1 error 
------------
Error:[4.10710529]
Pred:[1 0 1 1 1 1 0 1]
True:[0 1 1 0 1 0 1 0]
94 + 12 = 189
in these 1000 epochs, there's  998 error 
------------
Error:[3.67760383]
Pred:[1 0 0 0 0 1 0 0]
True:[1 0 1 1 0 1 0 0]
82 + 98 = 132
in these 1000 epochs, there's  991 error 
------------
Error:[3.78018724]
Pred:[0 1 0 0 0 0 0 0]
True:[1 0 1 0 1 0 1 0]
58 + 112 = 64
in these 1000 epochs, there's  967 error 
------------
Error:[2.83854546]
Pred:[0 1 1 1 0 1 0 1]
True:[0 1 0 0 0 1 0 1]
58 + 11 = 117
in these 1000 epochs, there's  800 error 
------------
Error:[1.03459131]
Pred:[1 0 0 0 1 1 0 0]
True:[1 0 0 0 1 1 0 0]
36 + 104 = 140
in these 1000 epochs, there's  422 error 
------------
Error:[1.10216843]
Pred:[0 1 1 1 0 1 0 0]
True:[0 1 1 1 0 1 0 0]
61 + 55 = 116
in these 1000 epochs, there's   58 error 
------------
Error:[0.62922836]
Pred:[0 1 0 0 0 1 0 0]
True:[0 1 0 0 0 1 0 